# Assistants API - Function Calling

In [12]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv
import os
import requests

_ : bool = load_dotenv(find_dotenv()) # read local .env file
openai_key = os.environ["OPENAI_API_KEY"]


In [13]:
client : OpenAI = OpenAI()

In [14]:
key = os.environ["FMP_API_KEY"]

def income_statment(period):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/0000320193?period={period}&apikey={key}"
    res = requests.get(url)
    return json.dumps(res.json())

def balance_sheet_statment(period):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/0000320193?period={period}&apikey={key}"
    res = requests.get(url)
    return json.dumps(res.json())

print(income_statment("annual"))

[{"date": "2023-09-30", "symbol": "AAPL", "reportedCurrency": "USD", "cik": "0000320193", "fillingDate": "2023-11-03", "acceptedDate": "2023-11-02 18:08:27", "calendarYear": "2023", "period": "FY", "revenue": 383285000000, "costOfRevenue": 214137000000, "grossProfit": 169148000000, "grossProfitRatio": 0.4413112958, "researchAndDevelopmentExpenses": 29915000000, "generalAndAdministrativeExpenses": 0, "sellingAndMarketingExpenses": 0, "sellingGeneralAndAdministrativeExpenses": 24932000000, "otherExpenses": -565000000, "operatingExpenses": 54847000000, "costAndExpenses": 268984000000, "interestIncome": 3750000000, "interestExpense": 3933000000, "depreciationAndAmortization": 11519000000, "ebitda": 125820000000, "ebitdaratio": 0.3282674772, "operatingIncome": 114301000000, "operatingIncomeRatio": 0.2982141227, "totalOtherIncomeExpensesNet": -565000000, "incomeBeforeTax": 113736000000, "incomeBeforeTaxRatio": 0.2967400237, "incomeTaxExpense": 16741000000, "netIncome": 96995000000, "netIncom

In [15]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [16]:


assistant = client.beta.assistants.create(
    instructions="financial analetics specialist",
    model="gpt-3.5-turbo-1106",
    tools=[{
        "type": "function",
        "function": {
          "name": "income_statment",
          "description": "Get income statment quater or annual as per parameter or desire query",
          "parameters": {
              "type": "object",
              "properties": {
                  "period": {"type": "string", "description": "income statment quater or annual"},
              },
              "required": ["period"]
          }
        }
    }, {
        "type": "function",
        "function": {
            "name": "balance_sheet_statment",
            "description": "Get balance sheet statment quater or annual as per parameter or desire query",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {"type": "string", "description": "balance sheet statment quater or annual"},
                },
                "required": ["period"]
            }
        }
    }
    ]
)

dict(assistant)

{'id': 'asst_CHhjbGRXR1pgjXVa6QthxZHZ',
 'created_at': 1702623853,
 'description': None,
 'file_ids': [],
 'instructions': 'financial analetics specialist',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='income_statment', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'income statment quater or annual'}}, 'required': ['period']}, description='Get income statment quater or annual as per parameter or desire query'), type='function'),
  ToolFunction(function=FunctionDefinition(name='balance_sheet_statment', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'balance sheet statment quater or annual'}}, 'required': ['period']}, description='Get balance sheet statment quater or annual as per parameter or desire query'), type='function')]}

In [17]:

thread = client.beta.threads.create()

dict(thread)


{'id': 'thread_d0v5ESugtkuVl0oSwZ5rq3Ie',
 'created_at': 1702623879,
 'metadata': {},
 'object': 'thread'}

In [18]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Kindly show quater income statemnet."
)
dict(message)

{'id': 'msg_1gb6TsmJws2Ezj0F0jhk1BaK',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Kindly show quater income statemnet.'), type='text')],
 'created_at': 1702623887,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_d0v5ESugtkuVl0oSwZ5rq3Ie'}

In [19]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
dict(run)

{'id': 'run_RIpRaicf32J2ACmPZQfBqgFS',
 'assistant_id': 'asst_CHhjbGRXR1pgjXVa6QthxZHZ',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702623903,
 'expires_at': 1702624503,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'financial analetics specialist',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_d0v5ESugtkuVl0oSwZ5rq3Ie',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='income_statment', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'income statment quater or annual'}}, 'required': ['period']}, description='Get income statment quater or annual as per parameter or desire query'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='balance_sheet_statment', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'de

In [20]:
available_functions = {
    "income_statment": income_statment,
    "balance_sheet_statment": balance_sheet_statment
}

In [21]:
import time

while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)

    if runStatus.status == "requires_action":
        
        
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    response = function_to_call(**function_args)
                    tool_outputs.append({"tool_call_id": toolcall.id,
                                  "output": response})
                  
                    
                  
            print(tool_outputs,">>>>>") 

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break




[{'tool_call_id': 'call_A7bP8vgGoTqqeGpNV4kR2A08', 'output': '[{"date": "2023-09-30", "symbol": "AAPL", "reportedCurrency": "USD", "cik": "0000320193", "fillingDate": "2023-11-03", "acceptedDate": "2023-11-02 18:08:27", "calendarYear": "2023", "period": "FY", "revenue": 383285000000, "costOfRevenue": 214137000000, "grossProfit": 169148000000, "grossProfitRatio": 0.4413112958, "researchAndDevelopmentExpenses": 29915000000, "generalAndAdministrativeExpenses": 0, "sellingAndMarketingExpenses": 0, "sellingGeneralAndAdministrativeExpenses": 24932000000, "otherExpenses": -565000000, "operatingExpenses": 54847000000, "costAndExpenses": 268984000000, "interestIncome": 3750000000, "interestExpense": 3933000000, "depreciationAndAmortization": 11519000000, "ebitda": 125820000000, "ebitdaratio": 0.3282674772, "operatingIncome": 114301000000, "operatingIncomeRatio": 0.2982141227, "totalOtherIncomeExpensesNet": -565000000, "incomeBeforeTax": 113736000000, "incomeBeforeTaxRatio": 0.2967400237, "incom